In [0]:
%%sh
aws s3 ls s3://b2c-prod-bigdata-mps/interface/MPS_BRANDED_KPI_V2/version=2.1.0/range_type=WEEK/




In [0]:

branded_kpi = "s3://b2c-prod-bigdata-mps/interface/MPS_MARKET_BENCHMARK_AGG_V2/version=2.1.0/range_type=WEEK/date=2021-01-09/"
spark.read.parquet(branded_kpi).createOrReplaceTempView("test")
spark.sql("SELECT genre_id, count(1) as co FROM test where country_code = 'US' group by genre_id order by co desc  limit 100 ").show(100, False)


In [0]:

bin_path = "s3://b2c-prod-bigdata-mps/interface/MPS_BIN_KPI_V2/version=2.0.0/range_type=WEEK/date=2021-05-08/"
attribute_kpi =  "s3://b2c-prod-bigdata-mps/interface/MPS_ATTRIBUTE_KPI_V2/version=2.0.0/range_type=WEEK/date=2021-05-08/"
branded_kpi = "s3://b2c-prod-bigdata-mps/interface/MPS_BRANDED_KPI_V2/version=2.0.0/range_type=WEEK/date=2021-05-08/"
market_benchmark = "s3://b2c-prod-bigdata-mps/interface/MPS_MARKET_BENCHMARK_AGG/version=2.0.0/range_type=WEEK/date=2021-05-08/"
msov = "s3://b2c-prod-bigdata-mps/interface/MPS_MONETIZATION_SOV/version=1.1.0/range_type=WEEK/date=2021-05-08/"
rating_kpi = "s3://b2c-prod-bigdata-mps/interface/MPS_RATING_KPI/version=1.1.0/range_type=WEEK/date=2021-05-08/"
total_downloads = "s3://b2c-prod-bigdata-mps/interface/MPS_TOTAL_DOWNLOADS/version=1.1.0/range_type=WEEK/date=2021-05-08/"
tmp= "s3://b2c-prod-bigdata-mps/interface/MPS_RATING_KPI/version=1.1.0/range_type=WEEK/date=2021-05-08/"


import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list
    


def get_date_granularity_mapping_list(begin_date, end_date):
    date_granularity_mapping_list = {
        "weekly": get_date_list(begin_date, end_date, "W-SAT")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2021-01-09", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-04-02", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
# range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}

# App IQ need to check



for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:        
        branded_kpi =  "s3://b2c-prod-bigdata-mps/interface/MPS_MARKET_BENCHMARK_AGG_V2/version=2.1.0/range_type=WEEK/date={date}/".format(date=date)
        spark.read.parquet(branded_kpi).createOrReplaceTempView("test")
        result = spark.sql("SELECT date, count(1)  FROM test group by date order by date desc ").collect()
        # result = spark.sql("SELECT date, count(distinct unified_product_id,country_code,device_code,market_code,granularity,date,genre_id)  FROM test group by date order by date desc ").collect()
        print(date, result[0][1])


# branded kpi
# spark.sql("SELECT date, count(distinct unified_product_id, country_code,genre_id,device_code,market_code,granularity,date)  FROM test where country_code = 'WW' group by date order by date desc ").show(10)
# spark.sql("SELECT date, count(1) FROM test  where country_code = 'WW' group by date order by date desc ").show(10)
# spark.read.parquet(market_benchmark).createOrReplaceTempView("test") 
# spark.sql("SELECT * FROM test where country_code='WW' and date = '2021-05-08' and unified_product_id = 1000600000699388 limit 10 ").show(100)

# market_benchmark
# spark.sql("SELECT date, count(distinct unified_product_id,country_code,genre_id,device_code,market_code,granularity,date)  FROM test group by date order by date desc ").show(10)
# spark.sql("SELECT * FROM test where country_code='CN' and date = '2021-05-08' and unified_product_id = 1000600000619545 limit 10 ").show(100)
# spark.sql("SELECT * FROM test where country_code='WW' and date = '2021-05-08' and unified_product_id = 1000600000567541 limit 10 ").show(100)



In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2013, 1, 1)
end_date = datetime(2014, 12, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


# DATE_GRANULARITY_MAPPINGLIST["monthly"].reverse()
# DATE_GRANULARITY_MAPPINGLIST["weekly"].reverse()
# DATE_GRANULARITY_MAPPINGLIST["daily"].reverse()

print (DATE_GRANULARITY_MAPPINGLIST["monthly"])
print (DATE_GRANULARITY_MAPPINGLIST["weekly"])
print (DATE_GRANULARITY_MAPPINGLIST["daily"])

In [0]:

spark.sparkContext.addPyFile("s3://b2c-prod-dca-bdp-data/BDP-PROD-APP-INT-QA/bdp/user_data/application_name=aa_data_test_app/application_code/version=latest/code/bdp_resources/usr/lib/spark/python/dependencies.zip")
from data_lib.api import spark_loader
loader = spark_loader(spark)
stu_df = loader.load_aurora("select date,count(1) from adl_mps_paid.fact_branded_mps_kpi_genre_au_v1 group by date order by  date desc").show(10)


In [0]:
%python
import snowflake.connector
def execute(cs, query):
    cs.execute(query)
    result = cs.fetchall()
    return result
warehouse = "AA_STORE_PRODUCT_LOCALIZED_SUMMARY_V1"
ctx = snowflake.connector.connect(
    user='htian',
    password='P2osNW6X8Xn&VCJ@',
    account='APPANNIE_AA_INT_PROD.us-east-1',
    role='DEV_DATA_TECH',
    session_parameters={
        'QUERY_TAG': 'load store adl sample {}',
    }
)
schema = "ADL_MASTER"
cs = ctx.cursor()
execute(cs, "use warehouse {};".format(warehouse))
execute(cs,"USE database aa_intelligence_production;")
execute(cs,"use role AA_UDW_WRITER;")
execute(cs,"USE schema {};".format(schema))
# print(execute(cs,sql_1))
print(execute(cs,"""ALTER TABLE ADL_MASTER.DIM_PRODUCT_LOCALIZED_DETAIL_V3_CLUSTER_BY_PRODUCT_KEY RENAME TO ADL_MASTER.DIM_PRODUCT_LOCALIZED_DETAIL_V1_CLUSTER_BY_PRODUCT_KEY """))